### Este é um código p[ara acessar a API do BCB
# Fonte: https://analisemacro.com.br/economia/indicadores/coletando-dados-do-banco-central-com-python/

https://wilsonfreitas.github.io/python-bcb/api.html

In [1]:
# Import packages

import pandas as pd

from bcb import sgs
from importlib import reload
import useful_functions

In [2]:


# Define the start date
start_date = '2001-01-31'

# Set the series to be downloaded - Source: https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries


data_series = {'RP_month':4639, # Get the primary result series - In R$ millions. Includes data from Central Government in Brazil: Federal Government + INSS + BACEN
               'IPCA_month_433': 433, # Monthly IPCA - Brazil's official inflation index
               'IPCA_month_13522_anual': 13522, # 12-months IPCA - Brazil's official inflation index
               'selic':432, # Selic (basic interest rate in Brazil) rate % per year
               'dbgg_%_pre':4537, # Gross Debt (% of GDP) - General Government (Federal, State and Municipal). Pre stands for a method used until 2007
               'dbgg_%_pos':13762, # Gross Debt (% of GDP) - General Government (Federal, State and Municipal). Pos stands for a method used since 2008
               'dbgg_R$_pre':4502, # Gross Debt (R$ millions) - General Government (Federal, State and Municipal). Pre stands for a method used until 2007               
               'dbgg_R$_pos':13761, # Gross Debt (R$ millions) - General Government (Federal, State and Municipal). Pos stands for a method used since 2008
               'dlgg_%':4536, # Net Debt (% of GDP) - General Government (Federal, State and Municipal)
               'dlgg_R$':4501,# Net Debt (R$ millions) - General Government (Federal, State and Municipal)
               'dlfg_%':4503, # Net Debt (% of GDP) - Federal Government + Central Bank
               'dlfg_R$':4468, # Net Debt (R$ millions) - Federal Government + Central Bank
               'dlspc_R$':4513, # Net Debt (%) - Federal Government + Central Bank
               'bal_payments_US$_M':22707,# Balance on goods - Balance of Payments - monthly - net
               'GDP_R$_12_months':4382,# 	GDP accumulated in the last 12 months - current prices (R$ million)
               'IR_US$':3546,# 	International reserves (US$ million) - Monthly
               
               } 

df_raw = sgs.get(data_series, start = start_date)




In [3]:
# Filter the data to get only the first day of each month
df_raw.index = pd.to_datetime(df_raw.index)

df_macro = df_raw[df_raw.index.day == 1].copy() # Create a copy of the raw data filtering only the first day of each month

In [4]:

# Rename the columns
df_macro = df_macro.rename(columns={
    'RP_month': 'eco_fiscal_result_month',
    'IPCA_month_433': 'eco_inflation_month',
    'IPCA_month_13522_anual': 'eco_inflation_12months',
    'selic': 'eco_interest_rate',
    'dbgg_%_pre': 'eco_gross_debt_gdp_pre',
    'dbgg_%_pos': 'eco_gross_debt_gdp_pos',
    'dbgg_R$_pre': 'eco_gross_debt_R$_pre',
    'dbgg_R$_pos': 'eco_gross_debt_R$_pos',
    'dlgg_%': 'eco_net_debt_gdp_%',
    'dlgg_R$': 'eco_net_debt_R$',
    'dlfg_%': 'eco_net_debt_gdp_%_federal_govt',
    'dlfg_R$': 'eco_net_debt_R$_federal_govt',
    'bal_payments_US$_M': 'eco_balance_payments_US$_M',
    'GDP_R$_12_months': 'eco_GDP_R$_12_months'
})

# Invert values from RP so that negative means deficit and positive means surplus
df_macro['eco_fiscal_result_month'] = df_macro['eco_fiscal_result_month'] * -1

# Compute the sum of last 12 months for each row
df_macro['eco_fiscal_result_12months'] = df_macro['eco_fiscal_result_month'].rolling(window=12).sum()

# Desired column order
column_order = ['eco_fiscal_result_month', 'eco_fiscal_result_12months', 'eco_inflation_month', 'eco_inflation_12months', 'eco_interest_rate', 'eco_gross_debt_gdp_pre', 'eco_gross_debt_gdp_pos', 'eco_gross_debt_R$_pre', 'eco_gross_debt_R$_pos', 'eco_net_debt_gdp_%', 'eco_net_debt_R$', 'eco_net_debt_gdp_%_federal_govt', 'eco_net_debt_R$_federal_govt', 'eco_balance_payments_US$_M', 'eco_GDP_R$_12_months']

# Reindex the DataFrame with the desired column order
df_macro = df_macro.reindex(columns=column_order)



In [6]:
# Convert the dataset to monthly frequency getting the last valid value of each month
df_macro_monthly = useful_functions.convert_df_to_monthly(df_macro,'%Y-%m-%d')
# Export the dataset
df_macro_monthly.to_csv('../data/df_macroeco.csv', index=True)
